In [1]:
import os
from transformers import pipeline, AutoTokenizer

/home/jovyan/conda_envs/metrics_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bleurt_model = "vaiibhavgupta/finetuned-bleurt-large"
mpnet_model = "lear-lab/short-answer-classification"
modernbert_model = "active-projects/itell-question-generation/results/modernbert_multirc"
bleurt_threshold = 0.7

In [2]:
import os
from pathlib import Path

# 1) Where am I right now?
print("cwd:", os.getcwd())

# 2) What’s in here?
for name in os.listdir("."):
    print(" •", name)

# 3) Check the path you’re using
model_path = "/home/jovyan/active-projects/itell-question-generation/results/modernbert_multirc"  # whatever you set it to
p = Path(model_path)
print(f"\nChecking '{model_path}':")
print(" exists? ", p.exists())
print(" is_dir?", p.is_dir())
print(" is_file?", p.is_file())


cwd: /home/jovyan/active-projects/itell-question-generation/src
 • multirc-v2-sequence-lengths-tobasum.ipynb
 • prompts
 • answer_probabilities.ipynb
 • compare_models_tobasum.py
 • compare_models_tobasum.ipynb
 • modernbert-vs-mpnet-vs-bleurt-tobasum.ipynb
 • eval-data.ipynb
 • cri_human_scoring_analysis_tobasum.ipynb
 • authentic-test-data
 • modernbert-vs-mpnet-vs-bleurt-tobasum-Copy1.ipynb
 • ingest-data
 • evaluate-q&a.ipynb
 • combined-eval.ipynb
 • __pycache__
 • eval
 • decoder-models
 • eval.ipynb
 • mpnet
 • aqag-irr.ipynb
 • .env
 • bleurt
 • longformer-training.ipynb
 • .ipynb_checkpoints
 • modernbert
 • modernbert-training-tobasum.ipynb
 • 3-compare-models.ipynb
 • modernbert-metrics-tobasum.ipynb

Checking './results/modernbert_multirc':
 exists?  False
 is_dir? False
 is_file? False


In [3]:
class Bleurt():
    model_name = bleurt_model
    threshold = bleurt_threshold

    def __init__(self):
        self.classifier = pipeline(
            "text-classification",
            model=self.model_name,
            device="cuda",
        )

    def __call__(self, candidate: str, reference: str) -> int:
        sequence = f"{candidate}[SEP]{reference}"

        result = self.classifier(sequence)
        score = result[0]["score"]

        return 1 if score > self.threshold else 0

In [4]:
class Mpnet():
    model_name = mpnet_model
    revision = "77b846ec4606bfcfdf913888d7f0ab51f977a579"

    def __init__(self):
        self.classifier = pipeline(
            "text-classification",
            model=self.model_name,
            revision=self.revision,
            device="cuda",
            )

    def __call__(self, candidate: str, reference: str) -> int:
        sequence = f"{candidate}</s>{reference}"

        result = self.classifier(sequence)
        label = result[0]["label"]

        return 1 if label == "correct_answer" else 0

In [8]:
from transformers import AutoTokenizer

class ModernBERT():
    model_name = modernbert_model

    def __init__(self):
        self.classifier = pipeline(
            "text-classification",
            model=self.model_name,
            device="cuda",
            )

    def __call__(self, candidate: str, reference: str) -> int:
        sequence = f"{candidate}</s>{reference}"

        result = self.classifier(sequence)
        label = result[0]["label"]

        return 1 if label == "correct_answer" else 0

In [16]:
mpnet = Mpnet()
bleurt = Bleurt()
modernBERT = ModernBERT()

Device set to use cuda
Device set to use cuda
Device set to use cuda


In [13]:
pipes = {
    "Mpnet_pipe": Mpnet(),
    "Bleurt_pipe": Bleurt(),
    "ModernBERT_pipe": ModernBERT()
}

def evaluate_all_models(dataset, models, label_key="labels"):
    texts = []
    true_labels = []
    preds_dict = {name: [] for name in models.keys()}

    for example in dataset["test"]:
        text = example["text"]
        texts.append(input_text)

        true_label = example[label_key]
        true_labels.append(true_label)

        for name, pipe in pipes.items():
            pred_label = pipe.predict(student_response, reference) 
            preds_dict[name].append(pred)

    results_dict = {
        "input_text": texts,
        "true_label": true_labels
    }

    for name, preds in preds_dict.items():
        results_dict[f"{name}_pred"] = preds

    return pd.DataFrame(results_dict)

# df_predictions = evaluate_all_models(ds, models)

Device set to use cuda
Device set to use cuda
Device set to use cuda


In [14]:
mpnet("This a strong answer to the question", "This is a reference answer to the question")  # Scored incorrect (0)

0

In [15]:
bleurt("This a strong answer to the question", "This is a reference answer to the question")  # Scored correct (1)

1